In [1]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np

In [7]:
compras = ['P01690', 'P01691']

In [2]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [17]:
def api_call_func(api_params: dict, compras: list[str]) -> list[dict]:

    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']


    fields_purchase_lines = ['product_id', 'price_unit_discounted', 'date_order', 'order_id']
    purchase_lines_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.name", "in", compras)]])
    purchase_lines_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [purchase_lines_ids], {'fields':fields_purchase_lines})


    return purchase_lines_json

In [21]:
api_params = api_params_func(True)
purchase_lines_json = api_call_func(api_params, compras)
purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)

In [22]:
purchase_lines_df

,purchase_line_id,product_id,purchase_cost_unit,purchase_date_order,purchase_order_id,purchase_order_name
0,10146,12432,66.465,2024-04-16 05:04:45,1704,P01691
1,10145,27848,8000.000,2024-04-15 18:30:25,1703,P01690


In [20]:
def purchase_lines_df_fun(purchase_lines_json: list[dict]) -> pd.DataFrame:

    data_purchase_lines = []

    for purchase in purchase_lines_json:
        new = {}
        new['purchase_line_id'] = purchase['id']
        new['product_id'] = purchase['product_id'][0]
        new['purchase_cost_unit'] = purchase['price_unit_discounted']
        new['purchase_date_order'] = purchase['date_order']
        new['purchase_order_id'] = purchase['order_id'][0]
        new['purchase_order_name'] = purchase['order_id'][1]

        data_purchase_lines.append(new)

    purchase_lines_df1 = pd.DataFrame(data_purchase_lines)
    purchase_lines_df1['purchase_date_order'] = pd.to_datetime(purchase_lines_df1['purchase_date_order'], format='%Y-%m-%d %H:%M:%S')
    purchase_lines_df1['purchase_line_id'] = purchase_lines_df1['purchase_line_id'].astype('Int64')
    purchase_lines_df1['purchase_order_id'] = purchase_lines_df1['purchase_order_id'].astype('Int64')

    purchase_lines_df = purchase_lines_df1.loc[purchase_lines_df1['product_id'].drop_duplicates().index]

    return purchase_lines_df

In [19]:
def complete_costo_df_fun(purchase_lines_df: pd.DataFrame, product_template_df: pd.DataFrame) -> pd.DataFrame:
    
    complete_costo_df = product_template_df.merge(purchase_lines_df, how='left', on='product_id')


    complete_costo_df.loc[(complete_costo_df['write_date'] > complete_costo_df['purchase_date_order']) | (complete_costo_df['purchase_date_order'].isna()), 'cost_type'] = 'REPS'
    complete_costo_df.loc[(complete_costo_df['write_date'] > complete_costo_df['purchase_date_order']) | (complete_costo_df['purchase_date_order'].isna()), 'cost_unit'] = complete_costo_df['product_template_cost']
    
    complete_costo_df.loc[complete_costo_df['purchase_date_order'] > complete_costo_df['write_date'], 'cost_type'] = 'Compra'
    complete_costo_df.loc[complete_costo_df['purchase_date_order'] > complete_costo_df['write_date'], 'cost_unit'] = complete_costo_df['purchase_cost_unit']
    
    complete_costo_df.loc[complete_costo_df['product_type'] == 'service', 'cost_type'] = 'Servicio'
    complete_costo_df.loc[complete_costo_df['product_type'] == 'service', 'cost_unit'] = pd.NA

    complete_costo_df['cost_unit'] = complete_costo_df['cost_unit'] * -1


    return complete_costo_df


In [20]:
def costo_func(test_db: bool = False) -> pd.DataFrame:
    
    api_params = api_params_func(test_db)
    purchase_lines_json, product_template_json = api_call_func(api_params)

    purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)
    product_template_df = product_template_df_fun(product_template_json)


    complete_costo_df = complete_costo_df_fun(purchase_lines_df, product_template_df)

    return complete_costo_df


## Pruebas

In [21]:
api_params = api_params_func()
purchase_lines_json, product_template_json = api_call_func(api_params)

purchase_lines_df = purchase_lines_df_fun(purchase_lines_json)
product_template_df = product_template_df_fun(product_template_json)

complete_costo_df = complete_costo_df = complete_costo_df_fun(purchase_lines_df, product_template_df)
